In [87]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, RepeatVector, TimeDistributed

### Defining parameters

In [88]:
data = 'train-mod.csv'
kval = 10
itertot = 40
sigma = 1.2
itergd = 300

### Calcula valor através da função escolhida

In [89]:
def transforminput(param, center):
    
    def rbfGaussiana(i, j, sigma):
        return np.exp(-(np.sum((param[i] - center[j])**2.0)**0.5) / sigma**2.0)
  
    def rbfMultiquadratica(i, j):
        return (np.sqrt(param[i]**2.0 + center[j]**2.0)/center[j])

    def rbfMultiquadraticaInversa(i, j):
        return (center[j]/np.sqrt(param[i]**2.0 + center[j]**2.0))
    
    newinput = np.zeros((len(param), len(center))).astype('float32')
    for i in range(len(param)):
        for j in range(len(center)):
            newinput[i,j] = rbfGaussiana(i, j, sigma)
    return newinput

In [90]:
def generatemodel(numparam):
    model = Sequential()
    model.add(Dense(1, input_dim=numparam, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [91]:
dataread = np.genfromtxt(data, delimiter=',')[1:,1:]

### Tratando entrada

In [92]:
alldata = []
for i in range(len(dataread)):
    if np.isnan(dataread[i,-2]):
        continue
    alldata.append(dataread[i])

alldata = np.asarray(alldata)

#dividing data

trainparam = alldata[:600,1:]
trainlabel = alldata[:600,0]
testparam = alldata[600:,1:]
testlabel = alldata[600:,0]

###############
#normalization#
###############

std = np.zeros((len(trainparam[0]))).astype('float32')
rata = np.zeros((len(trainparam[0]))).astype('float32')
trainparamnorm = np.zeros(np.shape(trainparam))
testparamnorm = np.zeros(np.shape(testparam))
for i in range(len(trainparam[0])):
    std[i] = np.std(trainparam[:,i])
    rata[i] = np.mean(trainparam[:,i])
    trainparamnorm[:,i] = (trainparam[:,i] - rata[i]) / std[i]
    testparamnorm[:,i] = (testparam[:,i] - rata[i]) / std[i]

### KMeans

In [93]:
kmean = np.zeros((kval, len(trainparamnorm[0])))
for i in range(kval):
    for j in range(len(kmean[0])):
        kmean[i,j] = random.uniform(min(trainparamnorm[:,j]),max(trainparamnorm[:,j]))
#looping of real algorithm
distmin = np.zeros((len(trainparamnorm)))
for i in range(itertot):
    print ('iteração para: ', i)
    for j in range(len(distmin)):
        #determine euclid distance
        distall = np.sum((trainparamnorm[j] - kmean)**2.0, axis=1)**0.5
        distmin[j] = np.argmin(distall)
#search new k mean
for j in range(kval):
    clust = []
    for k in range(len(distmin)):
        if distmin[k] == j:
            clust.append(trainparamnorm[k])
    if len(clust) > 0:
        kmean[j] = np.mean(np.asarray(clust), axis=0)

iteração para:  0
iteração para:  1
iteração para:  2
iteração para:  3
iteração para:  4
iteração para:  5
iteração para:  6
iteração para:  7
iteração para:  8
iteração para:  9
iteração para:  10
iteração para:  11
iteração para:  12
iteração para:  13
iteração para:  14
iteração para:  15
iteração para:  16
iteração para:  17
iteração para:  18
iteração para:  19
iteração para:  20
iteração para:  21
iteração para:  22
iteração para:  23
iteração para:  24
iteração para:  25
iteração para:  26
iteração para:  27
iteração para:  28
iteração para:  29
iteração para:  30
iteração para:  31
iteração para:  32
iteração para:  33
iteração para:  34
iteração para:  35
iteração para:  36
iteração para:  37
iteração para:  38
iteração para:  39


In [94]:
#tranform our input
newinput = transforminput(trainparamnorm, kmean)

print(trainlabel)

[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.

In [95]:
mod = generatemodel(kval)
mod.fit(newinput, trainlabel, batch_size=20, epochs=itergd, verbose=1, shuffle=True)

Epoch 1/300
600/600 [==============================] - 0s 253us/step - loss: 0.2547 - acc: 0.3683
Epoch 2/300
600/600 [==============================] - 0s 37us/step - loss: 0.2523 - acc: 0.4183
Epoch 3/300
600/600 [==============================] - 0s 38us/step - loss: 0.2502 - acc: 0.5100
Epoch 4/300
600/600 [==============================] - 0s 41us/step - loss: 0.2482 - acc: 0.5500
Epoch 5/300
600/600 [==============================] - 0s 42us/step - loss: 0.2464 - acc: 0.5867
Epoch 6/300
600/600 [==============================] - 0s 42us/step - loss: 0.2449 - acc: 0.6217
Epoch 7/300
600/600 [==============================] - 0s 43us/step - loss: 0.2435 - acc: 0.6417
Epoch 8/300
600/600 [==============================] - 0s 35us/step - loss: 0.2422 - acc: 0.6267
Epoch 9/300
600/600 [==============================] - 0s 45us/step - loss: 0.2411 - acc: 0.6333
Epoch 10/300
600/600 [==============================] - 0s 46us/step - loss: 0.2400 - acc: 0.6283
Epoch 11/300
600/600 [======

600/600 [==============================] - 0s 36us/step - loss: 0.2107 - acc: 0.6517
Epoch 85/300
600/600 [==============================] - 0s 35us/step - loss: 0.2105 - acc: 0.6533
Epoch 86/300
600/600 [==============================] - 0s 45us/step - loss: 0.2103 - acc: 0.6583
Epoch 87/300
600/600 [==============================] - 0s 37us/step - loss: 0.2101 - acc: 0.6583
Epoch 88/300
600/600 [==============================] - 0s 42us/step - loss: 0.2099 - acc: 0.6600
Epoch 89/300
600/600 [==============================] - 0s 36us/step - loss: 0.2097 - acc: 0.6600
Epoch 90/300
600/600 [==============================] - 0s 44us/step - loss: 0.2095 - acc: 0.6583
Epoch 91/300
600/600 [==============================] - 0s 34us/step - loss: 0.2094 - acc: 0.6600
Epoch 92/300
600/600 [==============================] - 0s 45us/step - loss: 0.2092 - acc: 0.6583
Epoch 93/300
600/600 [==============================] - 0s 49us/step - loss: 0.2090 - acc: 0.6600
Epoch 94/300
600/600 [===========

600/600 [==============================] - 0s 43us/step - loss: 0.2003 - acc: 0.6700
Epoch 168/300
600/600 [==============================] - 0s 42us/step - loss: 0.2002 - acc: 0.6683
Epoch 169/300
600/600 [==============================] - 0s 34us/step - loss: 0.2001 - acc: 0.6700
Epoch 170/300
600/600 [==============================] - 0s 36us/step - loss: 0.2001 - acc: 0.6700
Epoch 171/300
600/600 [==============================] - 0s 43us/step - loss: 0.2000 - acc: 0.6700
Epoch 172/300
600/600 [==============================] - 0s 37us/step - loss: 0.1999 - acc: 0.6667
Epoch 173/300
600/600 [==============================] - 0s 36us/step - loss: 0.1998 - acc: 0.6667
Epoch 174/300
600/600 [==============================] - 0s 36us/step - loss: 0.1998 - acc: 0.6667
Epoch 175/300
600/600 [==============================] - 0s 36us/step - loss: 0.1997 - acc: 0.6667
Epoch 176/300
600/600 [==============================] - 0s 39us/step - loss: 0.1996 - acc: 0.6667
Epoch 177/300
600/600 [=

600/600 [==============================] - 0s 39us/step - loss: 0.1957 - acc: 0.6767
Epoch 250/300
600/600 [==============================] - 0s 41us/step - loss: 0.1956 - acc: 0.6783
Epoch 251/300
600/600 [==============================] - 0s 36us/step - loss: 0.1956 - acc: 0.6783
Epoch 252/300
600/600 [==============================] - 0s 33us/step - loss: 0.1955 - acc: 0.6783
Epoch 253/300
600/600 [==============================] - 0s 38us/step - loss: 0.1955 - acc: 0.6767
Epoch 254/300
600/600 [==============================] - 0s 37us/step - loss: 0.1954 - acc: 0.6783
Epoch 255/300
600/600 [==============================] - 0s 37us/step - loss: 0.1954 - acc: 0.6783
Epoch 256/300
600/600 [==============================] - 0s 49us/step - loss: 0.1954 - acc: 0.6767
Epoch 257/300
600/600 [==============================] - 0s 37us/step - loss: 0.1954 - acc: 0.6767
Epoch 258/300
600/600 [==============================] - 0s 38us/step - loss: 0.1953 - acc: 0.6767
Epoch 259/300
600/600 [=

In [96]:
newinputtest = transforminput(testparamnorm, kmean)

In [97]:
lifeprob = mod.predict(newinputtest)

### Armazenando os valores previstos

In [98]:
binpred = np.zeros((len(lifeprob)))
for i in range(len(lifeprob)):
    if lifeprob[i] > 0.5:
        binpred[i] = 1.

### Calculando a acurácia

In [99]:
score = 0
for i in range(len(testlabel)):
    if binpred[i] == testlabel[i]:
        score += 1
accbin = float(score) / float(len(testlabel))

In [100]:
errorabs = abs(score-len(lifeprob))
error = np.sum(errorabs/len(testlabel), axis=0)

In [101]:
brierscore = 0
for i in range(len(testlabel)):
    brierscore += (testlabel[i] - lifeprob[i])**2.0
brierscore = brierscore / float(len(testlabel))

In [102]:
for i in range(len(testlabel)):
    print (lifeprob[i], "-", testlabel[i])

[0.22493514] - 0.0
[0.67203724] - 1.0
[0.32377267] - 1.0
[0.23652086] - 0.0
[0.20093644] - 0.0
[0.2752723] - 0.0
[0.7167618] - 1.0
[0.3369565] - 0.0
[0.22820407] - 1.0
[0.8002689] - 1.0
[0.24070299] - 0.0
[0.79457223] - 1.0
[0.42796806] - 0.0
[0.25976908] - 0.0
[0.22459081] - 0.0
[0.3935905] - 0.0
[0.59685326] - 0.0
[0.6172326] - 1.0
[0.2332291] - 0.0
[0.38868874] - 1.0
[0.86403656] - 1.0
[0.38109586] - 1.0
[0.5584983] - 1.0
[0.41471672] - 0.0
[0.22755075] - 0.0
[0.2274172] - 0.0
[0.38437125] - 1.0
[0.2861355] - 0.0
[0.32279634] - 1.0
[0.62056553] - 0.0
[0.21416947] - 0.0
[0.22699872] - 0.0
[0.3250595] - 0.0
[0.672452] - 1.0
[0.43120646] - 1.0
[0.24717444] - 0.0
[0.43647215] - 0.0
[0.2585711] - 0.0
[0.49479145] - 1.0
[0.58604026] - 1.0
[0.32388946] - 1.0
[0.23304611] - 1.0
[0.25313404] - 0.0
[0.44072244] - 0.0
[0.3844752] - 0.0
[0.3250595] - 0.0
[0.6357168] - 1.0
[0.22931096] - 0.0
[0.32287532] - 0.0
[0.27181053] - 0.0
[0.39789337] - 0.0
[0.24978846] - 0.0
[0.39565408] - 0.0
[0.2384520

In [103]:
print ("Accuracy: ", accbin, "Brier score:", brierscore[0])
print ("Error: ", error)

Accuracy:  0.7631578947368421 Brier score: 0.1710333
Error:  0.23684210526315788
